In [1]:
!pip install transformers
!pip install sentencepiece
!pip install bert
!pip install accelerate -U
!pip install evaluate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3744 sha256=78f9440a10c1bf4f448231352a19d3a1467c97f8da3ff161a4ee56dafb8e2558
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6779 sha256=3f38d3089a07a5c4d71940f0cc63f3f007a3c549c8d566c97cf28afc9469768e
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, TFDistilBertForMultipleChoice, AdamW
import tensorflow as tf
import torch

In [2]:
!pip install tensorflow==2.14

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertForMultipleChoice.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForMultipleChoice: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a d

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
training_data = pd.read_csv('train_data_t5_aug.csv')

In [6]:
testing_data = pd.read_csv('test_data.csv')

In [7]:
def finding_labels():
  labels = []
  for index, item in training_data.iterrows():
    if item['label_A'] == "stereotype":
      labels.append(0)
    elif item['label_B'] == "stereotype":
      labels.append(1)
    else:
      labels.append(2)
  return labels

In [8]:
label = finding_labels()

In [9]:
first, second = [], []
sentence_type = ['sentence_A', 'sentence_B', 'sentence_C']
for index, item in training_data.iterrows():
  question = item['context'][:-1] if item['context'][-1] == '.' else item['context']
  question = "The stereotypical reason for this statement: " + question + " is "
  first_sentence = [question for i in range(3)]
  first = first + first_sentence
  second_sentence = [f"{question} {item[i]}" for i in sentence_type]
  second.append(second_sentence)

second = sum(second, [])
tokenized_examples = tokenizer(first, second, truncation=True)
element = {k: [v[i : i + 3] for i in range(0, len(v), 3)] for k, v in tokenized_examples.items()}

In [10]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import tensorflow as tf


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="tf",
        )

        batch = {k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()}
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [11]:
import pandas as pd
from torch.utils.data import Dataset

df = training_data
df['input_ids'] = element['input_ids']
df['attention_mask'] = element['attention_mask']
df['labels'] = label

In [12]:
import datasets
from datasets import Dataset, DatasetDict

In [13]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

In [14]:
td_train = Dataset.from_pandas(df_train)
td_valid = Dataset.from_pandas(df_valid)

In [15]:
ds = DatasetDict()

ds['train'] = td_train
ds['validation'] = td_valid

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 5
total_train_steps = (len(ds["train"]) // batch_size) * num_train_epochs
optimizer, schedule = create_optimizer(init_lr=10e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [19]:
from transformers import TFAutoModelForMultipleChoice

model = TFAutoModelForMultipleChoice.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForMultipleChoice: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMultipleChoice from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a d

In [20]:
data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
tf_train_set = model.prepare_tf_dataset(
    ds["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    ds["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
model.compile(optimizer=optimizer)  # No loss argument!

In [22]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [23]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my_distilbert_ft_model_2",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/yinani24/my_distilbert_ft_model_2 into local empty directory.


Download file tf_model.h5:   0%|          | 8.74k/256M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/256M [00:00<?, ?B/s]

In [24]:
callbacks = [metric_callback, push_to_hub_callback]

In [25]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=4, callbacks=callbacks)

Epoch 1/4
4/4 [==============================] - 136s 31s/step - loss: 1.0818 - val_loss: 1.0445 - accuracy: 0.5000
Epoch 2/4
4/4 [==============================] - 111s 32s/step - loss: 0.9306 - val_loss: 0.9166 - accuracy: 0.4375
Epoch 3/4
4/4 [==============================] - 108s 31s/step - loss: 0.6289 - val_loss: 0.7981 - accuracy: 0.6250
Epoch 4/4
4/4 [==============================] - 108s 29s/step - loss: 0.3497 - val_loss: 0.7044 - accuracy: 0.5000


In [26]:
from transformers import AutoTokenizer, TFAutoModelForMultipleChoice

In [27]:
tokenizer_new = AutoTokenizer.from_pretrained("my_distilbert_ft_model_2")
model_new = TFAutoModelForMultipleChoice.from_pretrained("my_distilbert_ft_model_2")

Some layers from the model checkpoint at my_distilbert_ft_model_2 were not used when initializing TFDistilBertForMultipleChoice: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForMultipleChoice were not initialized from the model checkpoint at my_distilbert_ft_model_2 and are newly initialized: ['dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
data_store = pd.DataFrame(columns = ['id','question', 'bias_type','score','predicted_type','predicted_answer'])

In [29]:
bias = ['stereotype', 'unrelated', 'anti-stereotype']
for index, item in testing_data.iterrows():
  id = item['id']
  bias_type = item['bias_type']
  options = [item["sentence_A"], item["sentence_B"], item["sentence_C"]]
  type_ans = [item['label_A'], item['label_B'], item['label_C']]

  question = "Reason behind this statement: " + item['context'].split('.')[0] + " because "
  encoded_input = tokenizer_new([[question, options[0]],
                          [question, options[1]],
                          [question, options[2]]], return_tensors="pt", padding=True)

  inputs = {k: tf.expand_dims(v, 0) for k, v in encoded_input.items()}
  outputs = model(inputs)  # batch size is 1
  logits = outputs.logits

  # predicted_class = logits.argmax().item()
  predicted_class = tf.argmax(logits, axis=1).numpy()[0]
  predicted_answer = options[predicted_class]
  predicted_type = type_ans[predicted_class]
  score = 0

  if predicted_type == "anti-stereotype" or predicted_type == "unrelated":
    score = 1

  new_row = {'id': id, 'question' : question, 'bias_type': bias_type, 'score': score, 'predicted_type': predicted_type, 'predicted_answer': predicted_answer}
  data_store = data_store.append(new_row, ignore_index=True)
  # print(data_store)
  # if count == 0:
  #   break

<ipython-input-29-ffcf396a5e74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-29-ffcf396a5e74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-29-ffcf396a5e74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-29-ffcf396a5e74>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-29-ffcf396a5e74>:27: FutureWarning: The frame.append method is de

In [30]:
data_store.to_csv('output_distilbert_stereoset_ft_aug.csv',index=False)